In [21]:
# Skriptas sukuria zive duomenų analogą iš MIT duomenų, į sub-aplanką rec_dir įrašomi
# transformuoti EKG įrašai *.npy ir anotacijų failai *.json.
# Išeities duomenys MIT_BIH atsisiunčiami iš aplanko mit-bih-arrhythmia-database-1.0.0
# https://archive.physionet.org/physiobank/database/html/mitdbdir/mitdbdir.htm
# The recordings were digitized at 360 samples per second per channel with 11-bit
# resolution over a 10 mV range. In most records, the upper signal (channel 0) is
# a modified limb lead II (MLII), obtained by placing the electrodes on the chest.

# Iš MIT reaguojame tik į anotacijas N,L,R,e,j,V,E,S,A,a,J,F,Q, visas kitas ignoruojame.
# MIT2ZIVE duomenyse paliekame tik makroanotacijas: N,S,V,F,U. 

# Failai skaldomi į 3 dalis, recordId = 1,2,3, kad būtų panašaus ilgio, kauip Zive įrašai.

# Programos planas:
# Nuskaitome MIT-BIH ekg įrašą: channel = 0, skaitome mlvoltus
# Anotacijų mapingas į N,V,S,F,U
# Pakeičiame diskretizavimo dažnį iš 360 į 200 Hz
# Perskaičiuojame anotacijos vietas
# Padalijame įrašus į tris dalis
# Įrašome ekg ir anotacijas zive formatu į diską
# Suformuojame comments.csv failą, kuriame komentaruose nurodysime DS1, ar DS2 

# //////////////////////////////////////////////////////////////////////////////////////////////////////
#  
import pandas as pd
import numpy as np
import wfdb
# Waveform Database Software Package (WFDB) for Python
# https://physionet.org/content/wfdb-python/4.1.0/

import json
import sys
from pathlib import Path
from neurokit2 import signal_resample
from collections import Counter
import datetime

# from zive_util_vu import create_dir
from mit_bih_util import create_dir

def make_dict_array(atr_sample, atr_symbol):
    array_of_dict = []
    for sample, symbol in zip(atr_sample, atr_symbol):
        d = {'sample': sample, 'symbol': symbol}
        array_of_dict.append(d)
    return array_of_dict   

def print_dict_array(dict_array, incl = []):
    for d in dict_array:
        sample = d['sample']
        symbol = d['symbol']
        if bool(incl) == False:
            if (symbol != 'N'):
                print(f"Sample: {sample}, Symbol: {symbol}")    
        else:
            if (symbol in incl):
                print(f"Sample: {sample}, Symbol: {symbol}")     

def zive_read_file_1ch(filename):
    f = open(filename, "r")
    a = np.fromfile(f, dtype=np.dtype('>i4'))
    ADCmax=0x800000
    Vref=2.5
    b = (a - ADCmax/2)*2*Vref/ADCmax/3.10*1000
    ecg_signal = b - np.mean(b)
    return ecg_signal

def zive_read_df_rpeaks(db_path, file_name):
    file_path = Path(db_path, file_name + '.json')
    with open(file_path,'r', encoding="utf8") as f:
        data = json.loads(f.read())
    df_rpeaks = pd.json_normalize(data, record_path =['rpeaks'])
    return df_rpeaks

def find_f_l(sample, lst):
    last_s_idx = -1
    result = []

    for i in sample:
        if lst[i] == 's':
            last_s_idx = i
        elif lst[i] == 'f' and last_s_idx != -1 and i > last_s_idx:
            result.append((last_s_idx, i))
            last_s_idx = -1        
    return result

def print_noises_intervals(noises_intervals, fs):    
    for dict_obj in noises_intervals:
        start = dict_obj['startIndex']/fs
        start_min = start//60
        start_sec = start % 60

        finish = dict_obj['endIndex']/fs
        finish_min = finish//60
        finish_sec = finish % 60
        # print(f"lowerIndex: {lowerIndex} upperLevel: {upperLevel} start: {start_min:.2f} min {start_sec:.1f} sec finish: {finish_min:.2f} min {finish_sec:.1f} sec")    
        print(f"start: {start_min:.2f} min {start_sec:.1f} sec finish: {finish_min:.2f} min {finish_sec:.1f} sec")    
 
def get_noise_intervals(noise_pairs, atr_sample):    
    noise_intervals = []
    for elem in noise_pairs:
        lowerIndex = elem[0]
        start = atr_sample[lowerIndex]

        upperLevel = elem[1]
        end = atr_sample[upperLevel]

        dict_obj = {'startIndex': start, 'endIndex': end}
        noise_intervals.append(dict_obj)
    return noise_intervals
 

def get_noise_pairs(atr_symbol, ann_subtype):
    # Susitvarkome su `~`
    occurrences = [i for i in range(len(atr_symbol)) if atr_symbol[i] == '~']
    print(occurrences)
    atr_symbol_trs = atr_symbol.copy()

    # Paverčiame '~' į 's' (startIndex) arba 'f' (finish)
    for idx in occurrences:
        if ann_subtype[idx] == 1 or ann_subtype[idx] == 3:
            atr_symbol_trs[idx] = 's'
        elif ann_subtype[idx] == 0 or ann_subtype[idx] == 2:
            atr_symbol_trs[idx] = 'f'
        else:
            atr_symbol_trs[idx] = ''

    # Ištraukiame įrašų triukšmų anotacijas (pradžias ir pabaigas)
    noises_pairs = find_f_l(occurrences, atr_symbol_trs)

    return noises_pairs, atr_symbol_trs

def split_noise_intervals_into_3(noises_intervals, range_limits):
    
    range1_max = range_limits[0]
    range2_max = range_limits[1]
    range3_max = range_limits[2]

    range1 = []
    range2 = []
    range3 = []

    for interval in noises_intervals:
        if interval['startIndex'] <= range1_max:
            if interval['endIndex'] <= range1_max:
                range1.append(interval)
            else:
                range1.append({'startIndex': interval['startIndex'], 'endIndex': range1_max})
                if interval['endIndex'] <= range2_max:
                    range2.append({'startIndex': range1_max, 'endIndex': interval['endIndex']})
                else:
                    range2.append({'startIndex': range1_max, 'endIndex': range2_max})
                    if interval['endIndex'] <= range3_max:
                        range3.append({'startIndex': range2_max, 'endIndex': interval['endIndex']})
                    else:
                        raise ValueError('Interval overlaps range limits')
        elif interval['startIndex'] <= range2_max:
            if interval['endIndex'] <= range2_max:
                range2.append(interval)
            else:
                range2.append({'startIndex': interval['startIndex'], 'endIndex': range2_max})
                if interval['endIndex'] <= range3_max:
                    range3.append({'startIndex': range2_max, 'endIndex': interval['endIndex']})
                else:
                    raise ValueError('Interval overlaps range limits')
        elif interval['startIndex'] <= range3_max:
            if interval['endIndex'] <= range3_max:
                range3.append(interval)
            else:
                range3.append({'startIndex': interval['startIndex'], 'endIndex': range3_max})
                raise ValueError('Interval overlaps range limits')
        else:
            raise ValueError('Interval exceeds range limits')

        ranges = [range1, range2, range3]
        k = 0
        for r in ranges:
            # displacement = range_limits[k] - range_limits[0]
            displacement = 120370
            for i in range(len(r)):
                # r[i]['startIndex'] = int(r[i]['startIndex'] - k*displacement)
                r[i]['startIndex'] = int(r[i]['startIndex'])
                # r[i]['endIndex'] = int(r[i]['endIndex'] - k*displacement)
                r[i]['endIndex'] = int(r[i]['endIndex'])
            k += 1    
    return ranges




print("\nSukuriamas Zive duomenų analogas iš MIT duomenų")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI\\DUOM_2022_RUDUO'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/DUOMENU_TVARKYMAS_2023'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Aplankas su MIT-BIH duomenų rinkiniu
db_folder_mit = 'mit-bih-arrhythmia-database-1.0.0'

#  Aplankas, kur rašome MIT2ZIVE duomenis
db_folder_mit2zive = 'records_npy_tst'


# Užduodamas pacientų įrašų sąrašas
# Testavimui
records_nr = np.array([116])
# records_nr = np.array([124,209,228])

records_nr = np.array(
[100, 101, 103, 105, 106, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124,
200, 201, 202, 203, 205, 208, 209, 210, 212, 213, 214, 215, 219, 220, 221, 222, 223, 228, 230, 231,
232, 233, 234])

records_nr = np.array([232])

# MIT_BIH duomenų diskretizavimo dažnumas
fs_mit = 360


# ////////////////////////////////////////////////////////////////////////

# Nuoroda į MIT-BIH duomenų rinkinį
db_path_mit = Path(Duomenu_aplankas, db_folder_mit)

print("Bendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("MIT duomenų aplankas: ", db_folder_mit)
print("Pacientų įrašų sąrašas:\n",records_nr)
print("Aplankas, kur rašome MIT2ZIVE duomenis:\n", db_folder_mit2zive)

print("Pastaba: įrašai su nr. 102, 104, 107, ir 217 yra gauti iš pacientų su pacemakers ir yra praleisti")
print("207 paciento įrašas turi segmentus su ventricular flutter or fibrillation VF")
print("\nDiskretizavimo dažnis fs_mit: ", fs_mit)


# //////////////// Nurodomi MIT2ZIVE parametrai ////////////////////////////////


# MIT2ZIVE duomenų diskretizavimo dažnumas
fs_zive = 200 # diskretizavimo dažnumas

# Neignoruojamų anotacijų sąrašas
annot_list = ['N','L','R','e','j','A','a','J','S','V','E','F','Q','f','s']

# Failai pūpsnių makro anotacijų formavimui
annot_grouping = {
'N':'N','R':'N', 'L':'N', 'e':'N', 'j':'N', 'A':'S','a':'S', 'J':'S', 'S':'S', 'V':'V', 'E':'V', 'F':'F','Q':'U', 's':'s', 'f':'f' }


selected_beats = {'N':0, 'S':1, 'V':2, 'F':3, 'U':4}

# ////////////////////////////////////////////////////////////////////////

# Nuoroda į MIT2ZIVE duomenų rinkinį
# db_path_mit2zive = Path(Duomenu_aplankas, db_folder_mit2zive)

# Nuoroda į aplanką MIT2ZIVE EKG įrašams (.npy) ir anotacijoms (.json)
rec_dir = Path(Duomenu_aplankas, db_folder_mit2zive)

sant = fs_zive/fs_mit

print("\nAplankas transformuotiems duomenims: ", db_folder_mit2zive)

# Sukūriame aplanką transformuotiems iš MIT į Zive formatą duomenims
create_dir(rec_dir)

print("Diskretizavimo dažnis fs_zive: ", fs_zive)
print(f"fs_zive/fs_mit = {sant:.2f}")

print("Paliekamos įrašuose anotacijos:", annot_list)
print("Failas grupavimui į makroanotacijas:", annot_grouping)

# Noise Table





Sukuriamas Zive duomenų analogas iš MIT duomenų
OS in my system :  linux
Bendras Zive duomenų aplankas:  /home/kesju/DI/DUOMENU_TVARKYMAS_2023
MIT duomenų aplankas:  mit-bih-arrhythmia-database-1.0.0
Pacientų įrašų sąrašas:
 [232]
Aplankas, kur rašome MIT2ZIVE duomenis:
 records_npy_tst
Pastaba: įrašai su nr. 102, 104, 107, ir 217 yra gauti iš pacientų su pacemakers ir yra praleisti
207 paciento įrašas turi segmentus su ventricular flutter or fibrillation VF

Diskretizavimo dažnis fs_mit:  360

Aplankas transformuotiems duomenims:  records_npy_tst
Directory '/home/kesju/DI/DUOMENU_TVARKYMAS_2023/records_npy_tst' already exists
Diskretizavimo dažnis fs_zive:  200
fs_zive/fs_mit = 0.56
Paliekamos įrašuose anotacijos: ['N', 'L', 'R', 'e', 'j', 'A', 'a', 'J', 'S', 'V', 'E', 'F', 'Q', 'f', 's']
Failas grupavimui į makroanotacijas: {'N': 'N', 'R': 'N', 'L': 'N', 'e': 'N', 'j': 'N', 'A': 'S', 'a': 'S', 'J': 'S', 'S': 'S', 'V': 'V', 'E': 'V', 'F': 'F', 'Q': 'U', 's': 's', 'f': 'f'}


In [22]:

# Ciklas per pacientų įrašus
for record_nr in records_nr:


# ------------------------------------------ Nuskaitome įrašą ir jo atributus
# ---------------------------------------------------------------------------
    subject_path = f'{db_path_mit}/{record_nr}'
    # ic(subject_path)

    # Įrašo skaitymo variantai
    # sequence, fields = wfdb.rdsamp(subject_path, channels=[0])
    # record = wfdb.rdrecord('sample-data/a103l')

    # https://www.programmersought.com/article/28613723297/
    
    #  Variantas 1 - skaitmeninės reikšmės
    # record = wfdb.rdrecord(subject_path, sampfrom=1,
                                # channels=[0], physical=False)
    # sign_raw = record.d_signal[:,0]

    # Variantas 2 - fizinės reikšmės - perverstos į mV
    
    record = wfdb.rdrecord(subject_path, sampfrom=0,channels=[0], physical=True)
    signl_raw = record.p_signal[:,0]
    # print(sign_raw[:20])

    # https://wfdb.readthedocs.io/en/latest/wfdb.html

    len_signl_raw = signl_raw.shape[0]
    print("\n")
    print(f"Pacientas iš MIT: {record_nr}  Reikšmių: {len_signl_raw}")

    # Eilutės galimam filtravimui
    # sign_transf = sign_raw
    # sign_transf = signal_filter(signal=sign_raw, sampling_rate=360,
    #  lowcut=0.1, method="butterworth", order=5)

    # Nuskaitome originalaus įrašo anotacijas
    # https://wfdb.readthedocs.io/en/latest/wfdb.html 
    ann = wfdb.rdann(subject_path, 'atr', sampfrom=0, sampto=None, shift_samps=False)
    atr_sample_org = ann.sample
    atr_symbol_org = np.array(ann.symbol)
    print('len(atr_sample_org):', len(atr_sample_org))
    ann_subtype = ann.subtype
    print('len(ann.subtype):', len(ann.subtype))

    num = 100
    print('\natr_symbol_org:', atr_symbol_org[:num])
    print('\natr_sample_org:', atr_sample_org[:num])
    print('\nann_subtype:', ann_subtype[:num])

    noises_pairs_org, atr_symbol_trs = get_noise_pairs(atr_symbol_org, ann_subtype)
    print(noises_pairs_org)
    noises_intervals_org = get_noise_intervals(noises_pairs_org, atr_sample_org)
    print('noises_intervals_org:', noises_intervals_org)
    print_noises_intervals(noises_intervals_org, 360)
    
    print()
    dict_array = make_dict_array(atr_sample_org, atr_symbol_trs)
    print_dict_array(dict_array, incl = ['s', 'f'])


    print("\nAnotacijų pasiskirstymas originale:")
    smb_lst = Counter(atr_symbol_trs)
    print(dict(smb_lst))
    print()
    # smb_lst = Counter(atr_symbol)
    # print("\nOriginalių anotacijų pasiskirstymas:")
    # print(smb_lst)
    # print()
    # dict_array = make_dict_array(atr_sample,atr_symbol)
    # print_dict_array(dict_array)



# ------------------------------------ Resampling iš 360 Hz (MIT) į 200 Hz (Zive)
# -------------------------------------------------------------------------------
    signl = signal_resample(signl_raw, sampling_rate=fs_mit,
                              desired_sampling_rate=200, method="numpy")
    # https://neurokit2.readthedocs.io/en/latest/_modules/neurokit2/signal/signal_resample.html

    signl_len = signl.shape[0]
    print(f"Pacientas Zive rinkiniui: {record_nr}  Resamplintas. Reikšmių: {signl_len}")
    # print(signl[:20])

    # Resampliname anotacijų vietas
    atr_sample_rsp = (atr_sample_org*sant).astype(int)
    # pastaba: po resampling atsiranda tik atr_sample_rsp, kuris pakeičia
    # atr_sample_org. Tuo tarpu atr_symbol_org, ann_subtype išlieka tie patys

    # Surandame resamplinto įrašo triukšmų intervalus
    num = 100
    print('\natr_symbol_org:', atr_symbol_org[:num])
    print('\natr_sample_rsp:', atr_sample_rsp[:num])
    print('\nann_subtype:', ann_subtype[:num])

    noises_pairs, atr_symbol_trs = get_noise_pairs(atr_symbol_org, ann_subtype)
    noises_intervals = get_noise_intervals(noises_pairs, atr_sample_rsp)
    print('noises_intervals:', noises_intervals)
    print_noises_intervals(noises_intervals,200)

    print("\nAnotacijų pasiskirstymas po resampling:")
    smb_lst = Counter(atr_symbol_trs)
    print(dict(smb_lst))
    print()
    dict_array = make_dict_array(atr_sample_rsp,atr_symbol_trs)
    # print_dict_array(dict_array, incl = ['s', 'f'])

# ------------------------------ Anotacijas apvalome ir sukūriame makro anotacijas
    # Paliekame tik užduotas anotacijas iš annot_list, visas kitas (tarnybines)
    # išvalome, išskyrus `~`, jas išvalysime vėliau 
    # Tarnybinės yra: '+', ir visos kitos
    
    atr_sample_n = []
    atr_symbol_n = []
    for i in range(atr_sample_rsp.shape[0]):
        if atr_symbol_trs[i] not in annot_list:
            continue
        else:
            atr_sample_n.append(atr_sample_rsp[i])
            atr_symbol_n.append(atr_symbol_trs[i])
    atr_sample = np.array(atr_sample_n)
    atr_symbol = np.array(atr_symbol_n)

    print("\nAnotacijų pasiskirstymas po anotacijų valymo:")
    smb_lst = Counter(atr_symbol)
    print(dict(smb_lst))
    print()
    # dict_array = make_dict_array(atr_sample,atr_symbol)
    # print_dict_array(dict_array)

    # Anotacijas stambiname į makro anotacijas
    for i in range(len(atr_symbol)):
        if atr_symbol[i] in annot_grouping:
            atr_symbol[i] = annot_grouping[atr_symbol[i]]

    print("Makro anotacijų pasiskirstymas:")
    smb_lst = Counter(atr_symbol)
    print(dict(smb_lst))
    # print()
    # dict_array = make_dict_array(atr_sample, atr_symbol)
    # print_dict_array(dict_array)

     
# ------------------------------ Sudaliname įrašus ir anotacijas į n dalių ir įrašome į diską
# -------------------------------------------------------------------------------------------   
    n = 3 # dalių, į kurias daliname įrašus, skaičius

    # Įrašų ilgis po padalinimo
    len_sub_record = signl_len//n # will be assigned the value of the integer quotient
    # of dividing len_sign by n

    print('\nlen_sub_record:', len_sub_record)
    # print(n, signl_len, len_sub_record, len_sub_record*n)

    # Padaliname į 3 dalis triukšmų intervalus
    range_limits = [len_sub_record*i for i in range(1,4)]
    noise_intervals = split_noise_intervals_into_3(noises_intervals, range_limits)
    
    print("\nTriukšmai po sudalijimo:")
    print()
    print('noise_intervals:')
    for noise in noise_intervals:
        print(noise)

    # Paslenkame
    k = 0
    for r in noise_intervals:
        # displacement = range_limits[k] - range_limits[0]
        displacement = 120370
        for i in range(len(r)):
            r[i]['startIndex'] = r[i]['startIndex'] - k*displacement
            # r[i]['startIndex'] = int(r[i]['startIndex'])
            r[i]['endIndex'] = r[i]['endIndex'] - k*displacement
            # r[i]['endIndex'] = int(r[i]['endIndex'])
        k += 1    
    
    print("\nTriukšmai po paslinkimo:")
    print()
    print('noise_intervals:')
    for noise in noise_intervals:
        print(noise)

    dict_array = make_dict_array(atr_sample, atr_symbol)
    print_dict_array(dict_array, incl = ['s', 'f'])
    
    # signl_len = 10 # testavimui
    smb_lst_suminis = {'N': 0, 'S': 0, 'V': 0, 'F':0, 'U':0, 's':0, 'f':0 } # testavimui
    
    print("\nĮrašo ir anotacijų dalijimas į n dalių")
    
# -------------------------------------------------------------- Ciklas per n sub_records
    for sub_recId in range(n):
        print('\nsub_recId:', sub_recId, "Įrašo ilgis:", len_sub_record)
        lowerIndex = len_sub_record*sub_recId
        upperIndex = len_sub_record*(sub_recId+1) - 1
        # print('lowerIndex:', lowerIndex, 'upperIndex:', upperIndex, 'upperIndex-lowerIndex+1:', upperIndex-lowerIndex+1)

        # dividing signal array
        sub_record = np.empty(len_sub_record)
        sub_record = signl[lowerIndex:upperIndex+1]
        # print("len of sub_record:", len(sub_record))

        # Suformuojame padalinto EKG įrašo failo vardą (pvz. iš '100' padarome 10001.001) 
        rec_sub = '{:02d}'.format(sub_recId+1)
        rec_ext = '{:03d}'.format(sub_recId+1)
        file_name = str(record_nr) + rec_sub + '.' + rec_ext

        # Įrašome suformuotą sub_record į duomenų aplanką
        file_path = Path(rec_dir, file_name)
        with open(file_path, 'wb') as f:
            np.save(f, sub_record)
        # print('file_path:', file_path)
        
        # deviding annotation array
        sub_atr_sample = []
        sub_atr_symbol = []
        for i in range(len(atr_sample)):
            if (atr_sample[i] >= lowerIndex) & (atr_sample[i] < upperIndex):
                sub_atr_sample.append(atr_sample[i] - lowerIndex)
                sub_atr_symbol.append(atr_symbol[i])

        counter_obj = Counter(sub_atr_symbol) 
        annot_items = counter_obj.items()

        # testavimui
        print("Makro anotacijų pasiskirstymas:")
        dict_obj = dict(counter_obj)
        print(dict_obj)
        # print()
        for key in dict_obj:
            val = dict_obj[key]
            smb_lst_suminis[key] = smb_lst_suminis[key] + val
        print('smb_lst_suminis:', smb_lst_suminis)    
        # print()
        
# ---------------------------------- Paruošiame informaciją json failui ir įrašome
        recordingId = str(record_nr) + rec_sub

        rpeaks = []
        for idx in range(len(sub_atr_sample)):
            elem = {"sampleIndex": int(sub_atr_sample[idx]),
                    "annotationValue": sub_atr_symbol[idx]}
            rpeaks.append(elem)    
        print('rpeaks length:', len(rpeaks))

        json_data = {
            'recordingId': recordingId,
            "userId": str(record_nr),
            "rpeakAnnotationCounts": dict(annot_items),
            "noises": noise_intervals[sub_recId],
            "rpeaks": rpeaks 
        }

        # Įrašome json failą su įrašo atributais
        file_path = Path(rec_dir, file_name + '.json')
        with open(file_path, 'w') as f:
            json.dump(json_data, f)

    print('smb_lst_suminis:', smb_lst_suminis)    





Pacientas iš MIT: 232  Reikšmių: 650000
len(atr_sample_org): 1816
len(ann.subtype): 1816

atr_symbol_org: ['+' 'R' 'A' 'A' 'R' 'A' 'A' '~' 'R' 'A' 'R' 'A' 'R' 'A' 'R' 'A' 'A' 'R'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'R' 'R' 'R' 'A' 'A' 'R' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'R' 'R' 'A' 'A' 'R' 'A' 'A' 'A' 'R' 'R' 'A' 'A'
 'A' 'A' 'A' 'R' 'R' 'A' 'A' 'R' 'A' 'A' 'R' 'A' 'A' 'A' 'R' '~' 'R' 'R'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'R' '~' 'A' 'A' 'A' 'R' 'A' 'A'
 '~' 'R' 'A' 'A' 'A' 'A' 'A' 'R' 'A' 'A']

atr_sample_org: [   76   491   737  1000  1659  1903  2156  2550  2829  3066  3667  3909
  4511  4751  5341  5584  5855  6482  6721  6982  7232  7493  7754  8003
  8255  8941  9890 10295 10540 10796 11462 11704 11974 12237 12492 12747
 13003 13264 13526 13792 14058 14318 15317 15727 15966 16225 16918 17162
 17422 17684 18403 18892 19130 19391 19651 19905 20167 21128 21560 21805
 22066 22726 22965 23230 23887 24126 24384 24640 25352 25657 26100 26810
 27049 27306 27559 27821 28082

In [23]:
# ----------------------------------------------------------------------  SKAITYMO TESTAS 

print("\nKontrolinis skaitymas") 

# Kontrolinis  skaitymas

for sub_recId in range(n):
    rec_sub = '{:02d}'.format(sub_recId+1)
    rec_ext = '{:03d}'.format(sub_recId+1)
    file_name = str(record_nr) + rec_sub + '.' + rec_ext
   
    # Įrašas
    file_path = Path(rec_dir, file_name)
    # print('\nfile_path:', file_path)
    with open(file_path, "rb") as f:
        sign_loaded = np.load(f) 
        # print(sign_loaded[:10])
        # print('sign_loaded', sign_loaded.shape)
   
    # json
    # Nuskaitome gydytojo koreguotus EKG įrašo atributus iš įrašo json 
    df_rpeaks = zive_read_df_rpeaks(rec_dir, file_name)
    rpeaks_from_json = df_rpeaks['sampleIndex'].to_numpy()
    symbols_from_json = df_rpeaks['annotationValue'].to_numpy()
    # print(rpeaks_from_json[:20], symbols_from_json[:20])

    print("Makro anotacijų pasiskirstymas. Variantas iš json failo:")
    smb_lst = Counter(symbols_from_json)
    print(dict(smb_lst))
    print()
    # dict_array = make_dict_array(atr_sample,atr_symbol)
    # print_dict_array(dict_array)


print("\nPabaiga.............")



Kontrolinis skaitymas
Makro anotacijų pasiskirstymas. Variantas iš json failo:
{'N': 150, 'S': 455, 's': 8, 'f': 14}

Makro anotacijų pasiskirstymas. Variantas iš json failo:
{'N': 129, 'S': 464, 's': 2, 'f': 7}

Makro anotacijų pasiskirstymas. Variantas iš json failo:
{'N': 119, 'S': 463, 's': 2, 'f': 2}


Pabaiga.............


In [24]:
# Paruošiame šabloną comments.csv failui
# pvz.
# filename,userId,recordingId,nesutmp,quality,comment
# 1626934.963,0,0,"Kokybė puiki, išskyrus pabaigą"

dict_array = []
for record_nr in records_nr:
    # print(record_nr)
    for sub_recId in range(n):
        rec_sub = '{:02d}'.format(sub_recId+1)
        recordingId = str(record_nr) + rec_sub
        rec_ext = '{:03d}'.format(sub_recId+1)
        file_name = recordingId + '.' + rec_ext
        elem = {'filename':file_name,  'userId': str(record_nr), 'recId': recordingId, 'nesutmp': 0, 'quality': 0, 'comment': ''}
        dict_array.append(elem)

df = pd.DataFrame(dict_array)
print(df.head(5))
file_name = 'comments_sablonas.csv'
file_path = Path(rec_dir, file_name)
# df.to_csv(file_path, index=False)
# print("Šablonas įrašytas į: ", file_path)




    filename userId  recId  nesutmp  quality comment
0  23201.001    232  23201        0        0        
1  23202.002    232  23202        0        0        
2  23203.003    232  23203        0        0        
